# tutorial 24, create ec2 instance with (key pair & security) to login from local machine using ssh

In [1]:
import boto3
ec2_resourc=boto3.resource("ec2")


In [45]:
ec2_resourc.create_instances(ImageId='ami-03657b56516ab7912',
                             InstanceType="t2.micro",
                             MaxCount=1,
                             MinCount=1)

[ec2.Instance(id='i-04b69a02d164ee4ba')]

In [4]:
#create key pair
key_pair=ec2_resourc.create_key_pair(KeyName='key4')


In [48]:
key_value=key_pair.key_material

In [50]:
import io
with io.open("tutorial24.pem","w",encoding="utf-8")as f1:
    f1.write(str(key_value))
    f1.close()

In [51]:
#create security group
ec2_client=boto3.client("ec2")

In [54]:
ec2_client.create_security_group(Description='tutorial24 for ec2 ssh access',
      GroupName='tutorial24-sg')

{'GroupId': 'sg-0f1132f32dada677a',
 'ResponseMetadata': {'RequestId': 'cb8cffb5-d771-4588-b067-6c7e65f45284',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cb8cffb5-d771-4588-b067-6c7e65f45284',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '283',
   'date': 'Sat, 17 Oct 2020 17:04:51 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [58]:
#enable inbound rule for sg
ec2_client.authorize_security_group_ingress(
      GroupId='sg-0f1132f32dada677a',
      IpPermissions=[
          {
              'FromPort': 0,
              'IpProtocol': '-1',
              'IpRanges': [
                  {
                      'CidrIp': '0.0.0.0/0',
                      
                  },
              ],
              'ToPort': 65536,
          }
      ]
              )

{'ResponseMetadata': {'RequestId': 'eab6097e-8086-44fc-a2f6-10521fe321fa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'eab6097e-8086-44fc-a2f6-10521fe321fa',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '259',
   'date': 'Sat, 17 Oct 2020 17:12:31 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

# create ec2 instance with ssm agent ,security group , iampolicy and key pair

In [60]:
ec2_resourc.create_instances(ImageId='ami-03657b56516ab7912',
      InstanceType='t2.micro',
    KeyName='tutorial24',
    MaxCount=1,
      MinCount=1,
        SecurityGroupIds=[
          'sg-0f1132f32dada677a',
      ],
        IamInstanceProfile={
          'Name': 'ssm-role-ec2'
      },
    UserData="""
    #!/bin/bash
cd /tmp
sudo yum install -y https://s3.amazonaws.com/ec2-downloads-windows/SSMAgent/latest/linux_amd64/amazon-ssm-agent.rpm
sudo systemctl enable amazon-ssm-agent
sudo systemctl start amazon-ssm-agent
    """)

[ec2.Instance(id='i-025bab55dd7dd5512')]